In [13]:
from cassandra.cluster import Cluster
import pandas as pd

In [14]:
def pandas_factory(colnames, rows):    
    df= pd.DataFrame(rows, columns=colnames)    
    return df

In [15]:
cluster = Cluster(contact_points=['localhost'], port=9042)
session = cluster.connect()
session.set_keyspace('hotel')
session.row_factory = pandas_factory
#session.default_fetch_size = 10000000 #needed for large queries, otherwise driver will do pagination. Default is 50000.

# Consulta 1

1) Obtener un listado de los empleados del hotel, con todos sus datos.

In [16]:
rows_e = session.execute('SELECT * FROM empleado')
df_empleado=rows_e._current_rows
df_empleado.sort_values(by=['numreg'])

,numreg,cods,incorporacion,nombre,sueldo
4,1,1.0,1996-10-23,Luisa Blanco Baroja,1000.0
6,2,2.0,1996-10-23,Fernando Serrano Vázquez,1000.0
12,3,3.0,2000-01-01,Manuel Pérez Calo,900.0
7,4,4.0,2000-01-01,Ana Troncoso Calvo,900.0
0,5,1.0,2002-09-13,Alba Troncoso Calvo,NaN
9,6,5.0,2002-09-13,Jorge Alonso Alonso,NaN
8,7,5.0,2010-01-01,Fernando Soaje Alvarez,NaN
5,8,5.0,2010-01-01,Rosa Luigi Paz,1000.0
10,9,6.0,2010-01-01,Rafael Fuertes Cabrera,1100.0
1,10,NaN,2010-01-01,Antonio Sancho Sancho,1000.0


# Consulta 2

2) Obtener el nombre del jefe del servicio de "Restaurante".

In [17]:
rows_s = session.execute("SELECT * FROM servicio")
df_servicio=rows_s._current_rows
df_servicio#omitir

,cods,costeinterno,descripcion,numreg
0,5,50,restaurante,8
1,1,50,tintoreria,1
2,2,30,plancha,2
3,4,15,bar,4
4,6,25,floristeria,9
5,3,60,lavanderia,3


In [18]:
df_es = pd.merge(left=df_empleado, right=df_servicio, left_on='numreg', right_on='numreg')
display(df_es)#omitir
print('Consulta solicitada')#omitir
display(df_es.loc[df_es['descripcion'].str.contains('restaurante')]['nombre'])

,numreg,cods_x,incorporacion,nombre,sueldo,cods_y,costeinterno,descripcion
0,1,1.0,1996-10-23,Luisa Blanco Baroja,1000.0,1,50,tintoreria
1,8,5.0,2010-01-01,Rosa Luigi Paz,1000.0,5,50,restaurante
2,2,2.0,1996-10-23,Fernando Serrano Vázquez,1000.0,2,30,plancha
3,4,4.0,2000-01-01,Ana Troncoso Calvo,900.0,4,15,bar
4,9,6.0,2010-01-01,Rafael Fuertes Cabrera,1100.0,6,25,floristeria
5,3,3.0,2000-01-01,Manuel Pérez Calo,900.0,3,60,lavanderia


Consulta solicitada


1    Rosa Luigi Paz
Name: nombre, dtype: object

# Consulta 3

3) Obtener el nombre del jefe de "Jorge Alonso Alonso".

In [19]:
df_es2 = pd.merge(left=df_empleado, right=df_servicio, left_on='cods', right_on='cods')
sq=df_es2.loc[df_es2['nombre'].str.contains('Jorge Alonso Alonso')]['descripcion']
pd.merge(left=df_es, right=sq, left_on='descripcion', right_on='descripcion')['nombre']

0    Rosa Luigi Paz
Name: nombre, dtype: object

# Consulta 4

4) Obtener un listado de los empleados y los servicios a los que están asignados, exclusivamente para aquellos que tengan algún servicio asignado.

In [33]:
df_es.iloc[:,0:5]

,numreg,cods_x,incorporacion,nombre,sueldo
0,1,1.0,1996-10-23,Luisa Blanco Baroja,1000.0
1,8,5.0,2010-01-01,Rosa Luigi Paz,1000.0
2,2,2.0,1996-10-23,Fernando Serrano Vázquez,1000.0
3,4,4.0,2000-01-01,Ana Troncoso Calvo,900.0
4,9,6.0,2010-01-01,Rafael Fuertes Cabrera,1100.0
5,3,3.0,2000-01-01,Manuel Pérez Calo,900.0


# Consulta 5

5) Obtener el número de habitación, tipo y precio de las habitaciones que estén ocupadas en la actualidad (no tienen fecha de salida).

In [21]:
row_h=session.execute("SELECT * FROM habitacion")
row_f=session.execute("SELECT * FROM factura")
row_p=session.execute("SELECT * FROM precio")

df_habitacion=row_h._current_rows
df_factura=row_f._current_rows
df_precio=row_p._current_rows
#df_hf = pd.merge(left=df_habitacion, right=df_factura, left_on='numero', right_on='numero')
#df_hfp= pd.merge(left=df_hf, right=df_precio, left_on='tipo', right_on='tipo')

#df_hfp
display(df_habitacion)
display(df_factura)
display(df_precio)

,numero,bar,puedesupletoria,superficie,terraza,tipo
0,201,SI,NO,17,SI,doble
1,204,SI,SI,17,SI,doble
2,203,NO,SI,15,SI,individual
3,200,SI,SI,17,SI,doble
4,100,SI,SI,17,SI,doble
5,300,NO,SI,17,SI,doble
6,102,SI,SI,17,SI,individual
7,101,SI,SI,17,SI,doble
8,301,SI,SI,17,SI,doble
9,202,SI,NO,15,SI,individual


,codigof,dni,entrada,forma,numero,salida,supletoria,total
0,5,333333,2010-11-01,tarjeta,200,None,0,NaN
1,10,333333,2010-08-10,tarjeta,300,2010-08-13,0,270.0
2,16,666666,2010-08-19,efectivo,204,2010-08-21,0,212.0
3,13,999999,2010-05-01,efectivo,300,2010-05-05,0,380.0
4,11,888888,2010-08-15,tarjeta,102,2010-08-22,0,525.0
5,1,111111,2010-01-01,efectivo,100,2010-01-03,30,360.0
6,8,777777,2010-10-30,tarjeta,301,None,30,NaN
7,0,999999,2010-01-01,tarjeta,300,2010-01-05,0,220.0
8,2,111111,2010-06-01,efectivo,100,2010-06-03,30,180.0
9,4,111111,2010-11-02,efectivo,100,None,0,NaN


,tipo,precio
0,doble,90
1,individual,75


In [36]:
df_hf = pd.merge(left=df_habitacion, right=df_factura, left_on='numero', right_on='numero')
df_hfp= pd.merge(left=df_hf, right=df_precio, left_on='tipo', right_on='tipo')
df_hfp.loc[df_hfp['salida'].isnull()].iloc[:,[0,5,13]] 

,numero,tipo,precio
2,204,doble,90
3,200,doble,90
6,100,doble,90
11,300,doble,90
12,301,doble,90


# Consulta 6

6) Obtener el nombre y apellidos del cliente o clientes que más veces hayan estado en el hotel (no número total de noches, sino estancias diferentes: una persona que ha estado tres veces diferentes de una noche en el hotel habrá "estado" más veces que otra persona que ha estado una vez durante tres noches)

In [38]:
row_c=session.execute("SELECT * FROM cliente")
df_cliente=row_c._current_rows

df_cf = pd.merge(left=df_cliente, right=df_factura, left_on='dni', right_on='dni')
df_cf=df_cf.fillna({'Id': -1, 'total': 0})
df_cf_gb=df_cf.groupby(by=['nombre','apellidos'])
max_value=max(df_cf_gb.count()['total'])

df_master=df_cf_gb.count().reset_index()
df_6=df_master.loc[df_master['total']==max_value]
df_6.iloc[:,[0,1]]

,nombre,apellidos
0,Antonio,Aguirre
7,Virginia,Fernández Fernández
